In [1]:
try:
    from gwpy.timeseries import TimeSeries
except ModuleNotFoundError: 
    !pip install --quiet gwpy
    from gwpy.timeseries import TimeSeries

In [2]:
import os
import h5py as h5
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
import numpy as np
import pandas as pd

import sys
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [44]:
from sklearn.metrics import roc_auc_score,classification_report
from sklearn.preprocessing import label_binarize

Dataset Utility

In [36]:
def construct_dataframe(path,t_delta=None):
  name_data=[]
  starting_times_data=[]
  timeseries_data=[]
  type_data=[]
  for file in listdir(path):
    if file != '.ipynb_checkpoints':
        fout=h5.File(path+file,'r') 
    type_of_glitch=list(fout.keys())[0]
    dataset=fout[type_of_glitch]
    t = TimeSeries(dataset[()])
    ln=len(t)/2
    #if no t_delta is provided, the whole the timeseries are not cut 
    if t_delta==None:
      t_delta=ln
    name_data.append(file.split('.')[0].split('_')[0])
    type_data.append(type_of_glitch)
    starting_times_data.append(dataset.attrs['t0'])
    dataset_cut=dataset[()][int(ln-t_delta):int(ln+t_delta)]
    timeseries_data.append(dataset_cut)
    # intialise data of lists.

  data_Virgo = {'ID':name_data,
        'Type_Of_Glitch':type_data,
        'Starting_Times':starting_times_data,
        'TimeSeries':timeseries_data}
    # Create DataFrame
  df= pd.DataFrame(data_Virgo)
  return df

def check_remove_nanDF(dataframe, col='TimeSeries'):
    val=dataframe[col].values
    idx=[]
    for i in range(len(val)):
        if np.isnan(val[i])[0]==True:
            idx+=[i]
    print("removed timeseries:", len(idx))
    print(idx)
    return dataframe.drop(labels=idx,axis=0)




def build_dataset(path='content/sample_data/', idx=['_2', '_3', '_4', '_5', '_6', '_7']):
    df_LIGO=construct_dataframe(path+'Gravity_Spy/')
    df_LIGO=check_remove_nanDF(df_LIGO)
    for i in idx:
        df_tmp=construct_dataframe(path+'Gravity_Spy'+i+'/')
        df_tmp=check_remove_nanDF(df_tmp)
        df_LIGO=pd.concat([df_LIGO,df_tmp],axis=0)
    return df_LIGO

def build_npDataset(Dataset, norm=True):
    X,y=Dataset.T
    data=[]
    label=[]
    for element in X:
        data+= [element]
    for i in y:
        label+=[i]
    X_t=np.asarray(data)
    y_t=np.asarray(label)
    if norm:
        X_t=(X_t-X_t.mean(0))/X_t.std(0)
    return X_t,y_t

def numpy_glitch(df,norm=True):
    df.Type_Of_Glitch = pd.Categorical(df.Type_Of_Glitch)
    df['code']=df.Type_Of_Glitch.cat.codes
    return build_npDataset(df.drop(columns=['ID', 'Type_Of_Glitch', 'Starting_Times']).values)

def get_labels(df):
    return  sorted(list(set(datapoint[1] for datapoint in df.to_numpy())))

def remove_row_by_freq(df, col='Type_Of_Glitch', freq=50):
    return df[df.groupby(col)[col].transform('count').ge(freq)]


def split_stratified_into_train_val_test(df_input, stratify_colname='Type_Of_Glitch',
                                         frac_train=0.75, frac_val=0.15, frac_test=0.10,
                                         random_state=137):
    '''
    Splits a Pandas dataframe into three subsets (train, val, and test)
    following fractional ratios provided by the user, where each subset is
    stratified by the values in a specific column (that is, each subset has
    the same relative frequency of the values in the column). It performs this
    splitting by running train_test_split() twice.

    Parameters
    ----------
    df_input : Pandas dataframe
        Input dataframe to be split.
    stratify_colname : str
        The name of the column that will be used for stratification. Usually
        this column would be for the label.
    frac_train : float
    frac_val   : float
    frac_test  : float
        The ratios with which the dataframe will be split into train, val, and
        test data. The values should be expressed as float fractions and should
        sum to 1.0.
    random_state : int, None, or RandomStateInstance
        Value to be passed to train_test_split().

    Returns
    -------
    df_train, df_val, df_test :
        Dataframes containing the three splits.
    '''

    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)
    return df_train, df_val, df_test

In [9]:
df_LIGO=build_dataset()

removed timeseries: 4
[546, 635, 912, 1029]
removed timeseries: 31
[0, 30, 37, 99, 180, 186, 214, 217, 226, 238, 247, 256, 265, 275, 297, 304, 338, 387, 433, 447, 463, 506, 521, 567, 665, 695, 728, 776, 807, 814, 815]
removed timeseries: 24
[6, 27, 65, 79, 137, 144, 147, 184, 192, 204, 324, 361, 403, 408, 409, 411, 509, 575, 602, 606, 651, 697, 727, 742]
removed timeseries: 12
[20, 46, 47, 51, 120, 314, 353, 378, 412, 444, 456, 660]
removed timeseries: 21
[5, 18, 38, 100, 110, 169, 213, 239, 252, 263, 292, 313, 437, 455, 501, 573, 641, 657, 661, 672, 708]
removed timeseries: 17
[120, 142, 151, 179, 200, 279, 325, 338, 415, 446, 504, 508, 511, 552, 671, 687, 792]
removed timeseries: 27
[41, 48, 55, 59, 82, 93, 138, 179, 218, 260, 280, 294, 303, 369, 387, 416, 509, 598, 611, 635, 679, 683, 705, 718, 722, 828, 836]


In [10]:
df_LIGO['Type_Of_Glitch'].value_counts()

Blip                   1707
Koi_Fish                598
Low_Frequency_Burst     557
Power_Line              432
Low_Frequency_Lines     416
Scattered_Light         415
Extremely_Loud          287
Scratchy                247
Light_Modulation        221
Whistle                 144
1080Lines               140
No_Glitch               137
Tomte                   100
Repeating_Blips          79
1400Ripples              75
Chirp                    53
Air_Compressor           51
None_of_the_Above        40
Paired_Doves             15
Wandering_Line            6
Violin_Mode               2
Name: Type_Of_Glitch, dtype: int64

In [11]:
rem=True
if rem:
  df_LIGO=remove_row_by_freq(df_LIGO,freq=400)

In [12]:
df_LIGO['Type_Of_Glitch'].value_counts()

Blip                   1707
Koi_Fish                598
Low_Frequency_Burst     557
Power_Line              432
Low_Frequency_Lines     416
Scattered_Light         415
Name: Type_Of_Glitch, dtype: int64

In [13]:
labels=get_labels(df_LIGO)
labels

['Blip',
 'Koi_Fish',
 'Low_Frequency_Burst',
 'Low_Frequency_Lines',
 'Power_Line',
 'Scattered_Light']

In [47]:
labels_cod=np.arange(0,len(labels),1)
labels_cod

array([0, 1, 2, 3, 4, 5])

In [34]:
train_set, val_set, test_set = split_stratified_into_train_val_test(df_LIGO)

In [54]:
X,y=numpy_glitch(train_set)
X_val, y_val=numpy_glitch(val_set)
#test_set=pytorch_glitch(test_set)

In [55]:
y_val_bin=label_binarize(y_val, classes=labels_cod)

In [26]:
sample_rate=34864

In [39]:
y.shape

(3093,)

In [40]:
from sklearn.svm import SVC
svc = SVC(verbose=True)
svc.fit(X,y)

NameError: name 'y_test' is not defined

In [51]:
y_hat=svc.predict(X_val)

In [52]:
y_hat_bin=label_binarize(y_hat, classes=labels_cod)

In [56]:
print("ROC AUC Score:",roc_auc_score(y_val_bin,y_hat_bin, multi_class='ovr'))
print(classification_report(y_val,y_hat))

ROC AUC Score: 0.5
              precision    recall  f1-score   support

           0       0.41      1.00      0.59       256
           1       0.00      0.00      0.00        90
           2       0.00      0.00      0.00        83
           3       0.00      0.00      0.00        62
           4       0.00      0.00      0.00        65
           5       0.00      0.00      0.00        63

    accuracy                           0.41       619
   macro avg       0.07      0.17      0.10       619
weighted avg       0.17      0.41      0.24       619



/home/ecellini/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecellini/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecellini/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
from sklearn.ensemble import GradientBoostingClassifier

In [58]:
GBC=GradientBoostingClassifier()

In [ ]:
GBC.fit(X,y)


In [ ]:
y_hat=svc.predict(X_val)
y_hat_bin=label_binarize(y_hat, classes=labels_cod)

In [ ]:
print("ROC AUC Score:",roc_auc_score(y_val_bin,y_hat_bin, multi_class='ovr'))
print(classification_report(y_val,y_hat))